In [1]:
import pandas as pd
import numpy as np
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegressionCV
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from sklearn.mixture import DPGMM
from tabulate import tabulate

In [2]:
import sys
sys.path.append('../automated_model_creation') #We need to access the function file
from functions import *

In [3]:
df = pd.read_csv("../data/kaggle_data/ufc-master.csv")
df_upcoming = pd.read_csv('../data/kaggle_data/upcoming-event.csv')
len(df)



4783

In [4]:
num_upcoming_fights = len(df_upcoming)

In [5]:
df = df_upcoming.append(df)
len(df)

4794

In [6]:
df['date'] = pd.to_datetime(df['date'])

In [7]:
def return_finish_type(winner, finish):
    #print(winner, finish)
    #Why overcomplicate things?  We can just use a few if statements
    if winner == 'Red':
        #print("HI")
        if finish in ['U-DEC', 'S-DEC', 'M-DEC']:
            return ('Red - DEC')
        if finish in ['SUB']:
            return('Red - SUB')
        if finish in ['KO/TKO', 'DQ']:
            return('Red - KO/TKO')
    if winner == 'Blue':
        if finish in ['U-DEC', 'S-DEC', 'M-DEC']:
            return ('Blue - DEC')
        if finish in ['SUB']:
            return('Blue - SUB')
        if finish in ['KO/TKO', 'DQ']:
            return('Blue - KO/TKO')
        
    #Test for NaN
    if finish != finish:
        return('')
    
    if finish == 'Overturned':
        return('')
    
    
    return ('error')

In [8]:
#This calls for the power of lambda!
df['finish_type'] = df.apply(lambda x: return_finish_type(x['Winner'], x['finish']), axis=1)

In [9]:
finish_list = ['Red - DEC', 'Red - SUB', 'Red - KO/TKO', 'Blue - DEC', 'Blue - SUB', 'Blue - KO/TKO']

#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.

for f in range(len(finish_list)):
    mask = df['finish_type'] == finish_list[f]
    df['label'][mask] = f
    
#df["Winner"] = df["Winner"].astype('category')
#df = df[(df['Winner'] != 'Blue') | (df['Winner'] == 'Red') ]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

<ipython-input-9-af8f7542de83>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = f


In [10]:
df.rename(columns={'r_dec_odds': 'Red - DEC', 'r_sub_odds': 'Red - SUB', 'r_ko_odds': 'Red - KO/TKO',
                'b_dec_odds': 'Blue - DEC', 'b_sub_odds': 'Blue - SUB', 'b_ko_odds': 'Blue - KO/TKO'}, inplace=True)

In [11]:
df_label = df['label']
df_odds = df[finish_list]

In [12]:
df_train = df[num_upcoming_fights:]
odds_train = df_odds[num_upcoming_fights:]
label_train = df_label[num_upcoming_fights:]

df_test = df[:num_upcoming_fights]
odds_test = df_odds[:num_upcoming_fights]
label_test = df_label[:num_upcoming_fights]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

11
11
11
4783
4783
4783


In [13]:
weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)

C:\Users\matth\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [14]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,R_rev_bout,B_rev_bout,Red - DEC,Blue - DEC,Red - SUB,Blue - SUB,Red - KO/TKO,Blue - KO/TKO,finish_type,label
0,Dustin Poirier,Conor McGregor,-129,100,77.519380,100.000000,2021-07-10,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,550.0,700.0,575.0,2200.0,165.0,150.0,Red - KO/TKO,2.0
1,Gilbert Burns,Stephen Thompson,130,-162,130.000000,61.728395,2021-07-10,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,400.0,150.0,475.0,2500.0,525.0,250.0,Red - DEC,0.0
2,Tai Tuivasa,Greg Hardy,-130,105,76.923077,105.000000,2021-07-10,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,350.0,350.0,1600.0,1800.0,160.0,265.0,Red - KO/TKO,2.0
3,Irene Aldana,Yana Kunitskaya,-107,-115,93.457944,86.956522,2021-07-10,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,160.0,150.0,1200.0,1200.0,425.0,800.0,Red - KO/TKO,2.0
4,Sean O'Malley,Kris Moutinho,-1000,580,10.000000,580.000000,2021-07-10,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,400.0,1400.0,800.0,1600.0,-300.0,1200.0,Red - KO/TKO,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4778,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Blue - KO/TKO,5.0
4779,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red - KO/TKO,2.0
4780,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red - KO/TKO,2.0
4781,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red - DEC,0.0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,R_rev_bout,B_rev_bout,Red - DEC,Blue - DEC,Red - SUB,Blue - SUB,Red - KO/TKO,Blue - KO/TKO,finish_type,label
0,Islam Makhachev,Thiago Moises,-720,500,13.888889,500.000000,2021-07-17,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,-120.0,1200.0,350.0,1000.0,375.0,1400.0,,NaN
1,Marion Reneau,Miesha Tate,125,-145,125.000000,68.965517,2021-07-17,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,200.0,125.0,600.0,550.0,1100.0,1200.0,,NaN
2,Jeremy Stephens,Mateusz Gamrot,180,-220,180.000000,45.454545,2021-07-17,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,450.0,180.0,1600.0,1000.0,375.0,180.0,,NaN
3,Rodolfo Vieira,Dustin Stoltzfus,-220,180,45.454545,180.000000,2021-07-17,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,400.0,400.0,-105.0,1600.0,1000.0,475.0,,NaN
4,Gabriel Benitez,Billy Quarantillo,-165,145,60.606061,145.000000,2021-07-17,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,165.0,275.0,625.0,900.0,375.0,625.0,,NaN
5,Daniel Rodriguez,Preston Parsons,-350,270,28.571429,270.000000,2021-07-17,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,380.0,600.0,1100.0,650.0,-130.0,1000.0,,NaN
6,Amanda Lemos,Montserrat Conejo,-575,410,17.391304,410.000000,2021-07-17,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,110.0,750.0,500.0,1200.0,175.0,1600.0,,NaN
7,Khalid Taha,Sergey Morozov,-135,115,74.074074,115.000000,2021-07-17,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,165.0,215.0,600.0,1100.0,425.0,700.0,,NaN
8,Miles Johns,Anderson Dos Santos,-170,150,58.823529,150.000000,2021-07-17,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,120.0,350.0,1200.0,450.0,375.0,1100.0,,NaN
9,Francisco Figueiredo,Malcolm Gordon,-305,240,32.786885,240.000000,2021-07-17,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,275.0,600.0,450.0,700.0,150.0,900.0,,NaN


In [15]:
with open('../data/production_models_mov.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
#print(len(models))

#set the model choice automatically:
model_choice = 0
best_score = models[4][0]
for n in range(len(models[0])):
    print(n)
    if (float(models[4][n]) > float((best_score))):
        print("YES")
        best_score = models[4][n]
        model_choice = n

print(f"The model choice is {model_choice}")
print(models)

###REVERT THIS
#model_choice = 0

###


model = eval(models[1][model_choice])
features = eval(models[2][model_choice])
ev = eval(models[3][model_choice])
print()
print(model, features, ev)

0
The model choice is 0
[['model_2021_07_15'], ['LinearDiscriminantAnalysis()'], ['["R_Women\'s Featherweight_rank", \'B_Lightweight_rank\', \'R_Featherweight_rank\', \'R_Light Heavyweight_rank\', \'total_title_bout_dif\', \'R_losses\', \'B_win_by_Submission\', \'Blue - DEC\', \'B_draw\', \'R_Middleweight_rank\', \'B_avg_SIG_STR_pct\', "B_Women\'s Strawweight_rank", \'age_dif\', \'B_Middleweight_rank\', \'avg_sub_att_dif\', \'avg_td_dif\', \'B_avg_SUB_ATT\', "R_Women\'s Flyweight_rank", \'R_Pound-for-Pound_rank\', \'R_Reach_cms\', \'R_wins\', \'B_ev\', \'R_Weight_lbs\', \'sub_dif\', \'R_ev\', \'B_avg_TD_landed\']'], ['0'], ['6180']]

LinearDiscriminantAnalysis() ["R_Women's Featherweight_rank", 'B_Lightweight_rank', 'R_Featherweight_rank', 'R_Light Heavyweight_rank', 'total_title_bout_dif', 'R_losses', 'B_win_by_Submission', 'Blue - DEC', 'B_draw', 'R_Middleweight_rank', 'B_avg_SIG_STR_pct', "B_Women's Strawweight_rank", 'age_dif', 'B_Middleweight_rank', 'avg_sub_att_dif', 'avg_td_dif'

In [16]:
def display_bets(df, min_ev, finish_list, save_to_html=False):
    bet_list = []

    for index, row in df.iterrows():
        print()
        print()
        print(f"{row['t1_name']} (RED) vs. {row['t2_name']} (BLUE)")
        for f in finish_list:
            ev = get_bet_ev(row[f], row[f+'_prob'])
            if ev > min_ev:
                print(f"BET: {f}. ODDS: {row[f]}. PROB: {row[f+'_prob']}.  EV: {ev}")
                bet_list.append([row['t1_name'], row['t2_name'], f, row[f]])
                
    #print(bet_list)
    export_bets = pd.DataFrame(bet_list, columns=['Red', 'Blue', 'Bet', 'Odds'])
    if save_to_html:
        export_bets.to_csv('bets.csv', index=False)

In [17]:
def get_bets(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels,
             test_odds, finish_list, verbose=True, save_to_html=False):
    fighters_test = df[['R_fighter', 'B_fighter']]
    
    df_train = train_df[input_features].copy()
    df_test = test_df[input_features].copy()
    df_train = df_train.dropna()
    df_test = df_test.dropna()
        
    df_train = pd.get_dummies(df_train)
    df_test = pd.get_dummies(df_test)
    df_train, df_test = df_train.align(df_test, join='left', axis=1)    #Ensures both sets are dummified the same
    df_test = df_test.fillna(0)

    labels_train = train_labels[train_labels.index.isin(df_train.index)]
    odds_train = train_odds[train_odds.index.isin(df_train.index)] 
    #print(labels_train)
    print(len(df_train))
    labels_test = test_labels[test_labels.index.isin(df_test.index)]
    odds_test = test_odds[test_odds.index.isin(df_test.index)] 
    fighters_test = fighters_test[fighters_test.index.isin(df_test.index)]
    
    labels_train = labels_train.dropna()
    odds_train = odds_train[odds_train.index.isin(labels_train.index)]     
    df_train = df_train[df_train.index.isin(labels_train.index)] 
    
    
        
    input_model.fit(df_train, labels_train)
    
    probs = input_model.predict_proba(df_test)
    prob_label_list = []
    for f in finish_list:
        prob_label_list.append(f + '_prob')
    odds_test = np.array(odds_test)  
    fighters_test = np.array(fighters_test)
    #print(probs)
    #print()
    #print()
    prepped_test = list(zip(fighters_test[:, -2], fighters_test[:, -1], odds_test[:, 0], odds_test[:, 1],
                            odds_test[:, 2], odds_test[:, 3],odds_test[:, 4], odds_test[:, 5],
                            probs[:, 0], probs[:, 1],probs[:, 2], probs[:, 3],probs[:, 4], probs[:, 5]))
    ev_prepped_df = pd.DataFrame(prepped_test, columns=['t1_name', 't2_name'] + finish_list + prob_label_list)

    display_bets(ev_prepped_df, input_ev, finish_list, save_to_html=save_to_html)
    
    return(ev_prepped_df)

In [18]:
output_df = get_bets(model, features, ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, finish_list, verbose = True, save_to_html=True)

3355


Islam Makhachev (RED) vs. Thiago Moises (BLUE)
BET: Red - SUB. ODDS: 350.0. PROB: 0.3391430483707797.  EV: 52.61437176685085
BET: Red - KO/TKO. ODDS: 375.0. PROB: 0.34058293576831955.  EV: 61.776894489951786


Marion Reneau (RED) vs. Miesha Tate (BLUE)
BET: Red - KO/TKO. ODDS: 1100.0. PROB: 0.123294753898766.  EV: 47.95370467851919
BET: Blue - SUB. ODDS: 550.0. PROB: 0.3275979061042052.  EV: 112.93863896773337


Jeremy Stephens (RED) vs. Mateusz Gamrot (BLUE)
BET: Red - DEC. ODDS: 450.0. PROB: 0.28111341997384076.  EV: 54.61238098561242
BET: Blue - DEC. ODDS: 180.0. PROB: 0.3855296313088127.  EV: 7.94829676646755
BET: Blue - SUB. ODDS: 1000.0. PROB: 0.14161535154898117.  EV: 55.77688670387927


Rodolfo Vieira (RED) vs. Dustin Stoltzfus (BLUE)
BET: Red - DEC. ODDS: 400.0. PROB: 0.4066664185184721.  EV: 103.33320925923604


Gabriel Benitez (RED) vs. Billy Quarantillo (BLUE)
BET: Red - SUB. ODDS: 625.0. PROB: 0.19188102984622624.  EV: 39.113746638514016


Daniel Rodriguez (RED) vs.

In [19]:
output_df

,t1_name,t2_name,Red - DEC,Red - SUB,Red - KO/TKO,Blue - DEC,Blue - SUB,Blue - KO/TKO,Red - DEC_prob,Red - SUB_prob,Red - KO/TKO_prob,Blue - DEC_prob,Blue - SUB_prob,Blue - KO/TKO_prob
0,Islam Makhachev,Thiago Moises,-120.0,350.0,375.0,1200.0,1000.0,1400.0,0.203294,0.339143,0.340583,0.037261,0.021397,0.058322
1,Marion Reneau,Miesha Tate,200.0,600.0,1100.0,125.0,550.0,1200.0,0.240628,0.049340,0.123295,0.199950,0.327598,0.059189
2,Jeremy Stephens,Mateusz Gamrot,450.0,1600.0,375.0,180.0,1000.0,180.0,0.281113,0.023122,0.072550,0.385530,0.141615,0.096070
3,Rodolfo Vieira,Dustin Stoltzfus,400.0,-105.0,1000.0,400.0,1600.0,475.0,0.406666,0.297819,0.083121,0.105661,0.015070,0.091663
4,Gabriel Benitez,Billy Quarantillo,165.0,625.0,375.0,275.0,900.0,625.0,0.273354,0.191881,0.162048,0.186840,0.097384,0.088492
5,Daniel Rodriguez,Preston Parsons,380.0,1100.0,-130.0,600.0,650.0,1000.0,0.348530,0.074326,0.313509,0.125970,0.021843,0.115822
6,Amanda Lemos,Montserrat Conejo,110.0,500.0,175.0,750.0,1200.0,1600.0,0.286383,0.253191,0.222311,0.120162,0.087519,0.030434
7,Khalid Taha,Sergey Morozov,165.0,600.0,425.0,215.0,1100.0,700.0,0.349077,0.067928,0.113903,0.300021,0.041889,0.127181
8,Miles Johns,Anderson Dos Santos,120.0,1200.0,375.0,350.0,450.0,1100.0,0.347976,0.058102,0.139685,0.280427,0.069640,0.104169
9,Francisco Figueiredo,Malcolm Gordon,275.0,450.0,150.0,600.0,700.0,900.0,0.519749,0.078723,0.129075,0.147986,0.025284,0.099183
